In [ ]:
# Precompute phase:
# Go through dataset
# Grab list of mentions
# Query LLM to get new mentions and entities
# Save them in a good format

# Generation phase:
# Re-Generate data with a given seed
#import unsloth
#from unsloth import FastLanguageModel
# need C compiler...

STAGES = {0: "load dataset", 
          1: "generate mentions", 
          2: "verify entities", 
          3: "generate nif", 
          4: "generate candidates", 
          5: "generate descriptions", 
          6: "check for missing descriptions", 
          7: "generate aquaint nif", 
          -1: "generate_aquaint_nif"}
# 0, 4, 5, 6 are the ones we need to run
STAGE = STAGES[4]

In [2]:
#  from datasets import load_dataset

#  dataset = load_dataset("strombergnlp/ipm_nel", trust_remote_code=True)


In [3]:
from datasets import load_dataset
# Generate NIF dataset for AQUAINT
from pynif import NIFCollection

if STAGE == "generate_aquaint_nif":
    base_uri = "http://example.org/AQUAINT/"
    collection = NIFCollection(uri=base_uri)
    document_uri = base_uri + "document"
    nif_dataset_output_path = "./data/AQUAINT.ttl"
    if STAGE == "generate aquaint nif":
        dataset = load_dataset("boragokbakan/entity_disambiguation", "aquaint")
        test_data = dataset['test']
        test_idx = 1
        #print(test_data['candidates'][test_idx])
        #print(test_data['input'][test_idx])
        #print(test_data['id'][test_idx])
        #print(test_data['meta'][test_idx])#'left context', 'right context', 'mention'
        #print(test_data['answer'][test_idx])
        for i in range(len(test_data)):
            #entity = test_data['answer'][i]
            document_uri = base_uri + f"document{i}"
            # Add sentence (document) to context
            sentence_str = test_data['input'][i]
            phrases = []
            start_marker = "[START_ENT] "
            startoffset = sentence_str.index(start_marker)
            end_marker   = " [END_ENT]"
            # For each mention
            while start_marker in sentence_str:
                # Get the mention
                startoffset = sentence_str.index(start_marker)
                endoffset = sentence_str.index(end_marker, startoffset)
                # Get the mention
                mention = sentence_str[startoffset + len(start_marker):endoffset]
                sentence_str = sentence_str[:startoffset] + mention + sentence_str[endoffset + len(end_marker):]
                # Get the entity
                entity = test_data['answer'][i]
                #print(entity)
                # Add an entry
                phrases.append({
                    "beginIndex": startoffset,
                    "endIndex": startoffset + len(mention),
                    "taIdentRef": 'http://dbpedia.org/resource/' + entity.replace(" ", "_"),
                })

            context = collection.add_context(
                uri=document_uri,
                mention=sentence_str)
            
            for phrase in phrases:
                # Add a phrase to the context
                context.add_phrase(
                    beginIndex=phrase['beginIndex'],
                    endIndex=phrase['endIndex'],
                    taIdentRef=phrase['taIdentRef']
                )


        generated_nif = collection.dumps(format='turtle')
        #print(generated_nif)
        with open(nif_dataset_output_path, "w", encoding='utf-8') as dataset_file:
            dataset_file.write(generated_nif)

In [4]:
# Load NIF dataset
import utils
nif_data_folder_path = "/mnt/webscistorage/wf7467/agnos/data/"
nif_dataset_names = ["AIDA-Syn_mini_10_42.nif", "AIDA-Syn_mini_50_42.nif", "AIDA-Syn_mini_100_42.nif", "AQUAINT.ttl", "KORE_50_DBpedia.ttl", "ACE2004t.ttl", "MSNBCt.ttl", "aida_testc.ttl", "RSS-500.ttl", "AIDA-YAGO2-dataset.tsv_nif"]
nif_data_path_lst = [nif_data_folder_path + name for name in nif_dataset_names]
nif_data_path_idx = 0
# Just the name without the path
nif_dataset_name = nif_dataset_names[nif_data_path_idx]
nif_data_path = nif_data_path_lst[nif_data_path_idx]
# response_file_path is within _mappings.json
out_mappings_info_dir = nif_data_path + "_mappings/"

# Now we need to save our descriptions somewhere
out_descriptions_dir = nif_data_path + "_descriptions/"



if STAGE == "load dataset":
    # Load NIF dataset
    print("Loading NIF dataset...")
    # Load the dataset
    collection = utils.load_nif_dataset(nif_data_path)


In [5]:
#Go through all lines with specific regex criteria and output in a list
# Only relevant for homebrew dataset
if False:
    # This is only if we need to generate the context list because some NIF libraries don't take care of it
    # But PyNIF requires it to figure out what the context documents are
    print("NIF path: ",nif_data_path)
    hasContext_docEntities = utils.compute_nif_hasContext(nif_data_path)
    print(list(hasContext_docEntities))
    hasContext_docEntities = utils.compute_nif_hasContextKORE50(nif_data_path)
    print(list(hasContext_docEntities))
    hasContext_docEntities = utils.compute_nif_hasContextACE2004t(nif_data_path)
    print(list(hasContext_docEntities))
    hasContext_docEntities = utils.compute_nif_hasContextMSNBCt(nif_data_path)
    print(list(hasContext_docEntities))



In [6]:
if STAGE == "load dataset":
    print(collection)
    for context in collection.contexts:
        json_options = {}
        try:
            mapping_document_candidate_info = {}

            # See what can be called on context
            # print(dir(context))

            # document/context.mention = input sentence (1 document = 1 sentence)
            document = context.mention
            # If it's done already, continue to next

            # Document string will be used as key for all candidate information
            key = document

            mentions = [phrase.mention for phrase in context.phrases]
            entities = [phrase.taIdentRef.replace("http://dbpedia.org/resource/", "").replace("http://en.wikipedia.org/wiki/", "") for phrase in context.phrases]

            json_options = {}
            json_options['sentence'] = document
            json_options['mentions'] = {}
            json_options['mentions']['beginIndex'] = [phrase.beginIndex for phrase in context.phrases]
            json_options['mentions']['endIndex'] = [phrase.endIndex for phrase in context.phrases]
            json_options['mentions']['uris'] = entities
            json_options['mentions']['mention'] = mentions
        except Exception as e:
            print(f"Error processing document: {e}")
            continue

In [7]:
# Please install OpenAI SDK first: `pip3 install openai`
from openai import OpenAI
import json, os, glob
import os.path

# Load NIF dataset
import utils

if STAGE == "generate mentions":

    collection = utils.load_nif_dataset(nif_data_path)
    print(f"Finished loading dataset[{nif_data_path}]: {collection}")

    prompt_path = "/mnt/webscistorage/wf7467/agnos/prompt_generate_5_new_mention_minimized.txt"
    prompt_template = ""
    # Load the prompt template from a file
    with open(prompt_path, "r") as f:
        prompt_template =  f.read()

    # Load API key from file
    api_key_file = ".deepseek_api_key.txt"
    with open(api_key_file, "r") as f:
        DEEPSEEK_API_KEY = f.read().strip()

    # Init LLM client
    client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")


    out_dir = nif_data_path_lst[nif_data_path_idx] + "_options/"
    #out_dir = "/mnt/webscistorage/wf7467/agnos/data/AIDA-YAGO2-dataset.tsv_nif_options/"

    # Save mappings for input document to output document(s)
    out_mappings_info_dir = nif_data_path_lst[nif_data_path_idx] + "_mappings/"

    # Load all mappings
    mapping_keys = set()
    # List all files in out_mappings_info_dir
    # Get all JSON files in the directory
    json_mapping_files = glob.glob(os.path.join(out_mappings_info_dir, "*.json"))
    for mapping_file in json_mapping_files:
        with open(mapping_file, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            # Adapt to respective JSON structure
            mapping_keys.add(in_json.get("original_uri", ""))
            # { "original_uri": str, "response_file_path": str, "document": str}



    QUERY_LLM = True

    file_identifier = 0
    for context in collection.contexts:

        json_options = {}
        try:
            # See what can be called on context
            # print(dir(context))
            # context_actions = [context.add_phrase, context.beginIndex, context.endIndex, context.isContextHashBasedString, context.load_from_graph, context.mention, context.original_uri, context.phrases, context.sourceUrl, context.triples, context.turtle, context.uri]
            # context_action_names = ['add_phrase', 'beginIndex', 'endIndex', 'isContextHashBasedString', 'load_from_graph', 'mention', 'original_uri', 'phrases', 'sourceUrl', 'triples', 'turtle', 'uri']
            # for action, name in zip(context_actions, context_action_names):
            #     print(f"{name}: {action}")

            # Store following info: original_uri, document, response_file_path
            # document/context.mention = input sentence (1 document = 1 sentence)
            document = context.mention
            
            # Document string will be used as key for all candidate information
            key = context.original_uri

            # If it's done already, continue to next
            # Check mappings
            if key in mapping_keys:
                print(f"Found key[{key}] Skipping...")
                continue

            sorted_mentions_entities = sorted(
                [(phrase.mention, phrase.beginIndex, phrase.endIndex, phrase.taIdentRef) for phrase in context.phrases],
                key=lambda x: x[1]
            )
            # Only take the actual mention rather than the indices for displaying
            mentions = [mention[0] for mention in sorted_mentions_entities]
            # Also have the entities sorted appropriately - otherwise the indices are faulty
            entities = [entity[3].replace("http://dbpedia.org/resource/", "").replace("http://en.wikipedia.org/wiki/", "") for entity in sorted_mentions_entities]

            # json_options = {}
            # json_options['sentence'] = document
            # json_options['mentions'] = {}
            # json_options['mentions']['beginIndex'] = [phrase.beginIndex for phrase in context.phrases]
            # json_options['mentions']['endIndex'] = [phrase.endIndex for phrase in context.phrases]
            # json_options['mentions']['uris'] = entities
            # json_options['mentions']['mention'] = mentions

            print(mentions)
            print(entities)
            print(document)

            prompt = prompt_template
            prompt += "Original Text: \""+document + "\"\n" #Officer Korey Lankow was placed on administrative leave after leaving Jeg , a drug-sniffing dog , in his squad car outside DPS headquarters in Tucson for more than an hour on July 11 ."
            prompt += "Mentions: [\"" + "\", \"".join(mentions) + "\"]" +"\n"#["Tucson", "DPS"]
            prompt += "Wikipedia IDs: [\"" + "\", \"".join(entities) + "\"]"  + "\n"#["Tucson,_Arizona", "Department_of_Public_Safety"]
            # Original Text: "Samsung released a new phone in Seoul."  
            # Mentions: ["Samsung", "Seoul"]  
            # Wikipedia IDs: ["Samsung", "Seoul"]  
            #print(prompt)

            # Response output path
            out_response_path = out_dir + str(file_identifier) + "_response.json"
            while os.path.isfile(out_response_path):
                file_identifier += 1
                out_response_path = out_dir + str(file_identifier) + "_response.json"

            # Mappings output path
            out_mappings_file = out_mappings_info_dir + str(file_identifier) + "_mappings.json"
            # Add to redundancy check for skip logic
            mapping_keys.add(key)

            # Save mappings for input document to output document(s)
            # Save the mapping information
            out_mapping_info = {}
            # Save the original URI for the document (e.g. <https://aifb.kit.edu/conll/1025#char=0,1766>)
            out_mapping_info["original_uri"] = context.original_uri
            # Save the response file path, so we know where to look for the response
            out_mapping_info["response_file_path"] = out_response_path

            # Save the input document text
            out_mapping_info["document"] = document
            # Save the original mentions
            # out_mapping_info["mentions"] = mentions
            out_mapping_info["mention_entities"] = sorted_mentions_entities
            # Save the original entities
            # out_mapping_info["entities"] = entities
            # { "original_uri": str, "response_file_path": str, "document": str}
            out_mapping_info["prompt"] = prompt

            if QUERY_LLM:
                # Use the LLM to get the new mentions
                # Dump the mapping JSON to a file for easy retrieval
                with open(out_mappings_file, "w") as out_file:
                    json.dump(out_mapping_info, out_file, indent=4)
                # Open a new connection for each document

                response = client.chat.completions.create(
                    model="deepseek-chat",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant, an expert at generating new mentions for entity linking tasks."},
                        {"role": "user", "content": prompt},
                    ],
                    stream=False,
                    max_tokens=8000
                )
                # Save the response to a file
                #print(response.choices[0].message.content)
                with open(out_response_path, "w") as f:
                    f.write(response.choices[0].message.content)
                    print("Saved: ", out_response_path)

            #break
        except Exception as e:
            print(f"Error processing document: {e}")
            continue
        file_identifier += 1

In [8]:
import utils
import sparqlutils
import candgenutils
import glob, os, json

print(utils.get_json_from_response_file("/mnt/webscistorage/wf7467/agnos/data/AIDA-YAGO2-dataset.tsv_nif_options/176_response.json")['mentions'])
# yay, we can extract the new dataset stuff!
# Now, what needs to be done?
# 0. Verify that all wikipedia IDs are correct (and actually exist by doing a DBpedia lookup for description and type)
# 1. Get the new mentions and transform them into sentences by replacing {0} etc. with appropriate position mention
# After all have run through, transform into NIF datasets (to make downstream processing uniform)
# 2. Generate candidates for the new mentions (DBpedia Lookup or BLINK)
# 3. Generate types for each candidate (DBpedia Lookup? Or SPARQL query to DBpedia endpoint)
# 4. Query descriptions (SPARQL query via DBpedia endpoint)

def get_info_from_response_json(key, response_json, response_json_filepath):
    # Get the wanted information from the response
    info = response_json.get(key, None)
    if info == []:
        raise ValueError(f"No [{key}] found in [{response_file_path}] for [{response_json}]")
    return info


if STAGE == "verify entities":
    prefix_dbpedia = "http://dbpedia.org/resource/"
    # Load mappings
    # Get all JSON files in the directory
    json_mapping_files = glob.glob(os.path.join(out_mappings_info_dir, "*.json"))
    for mapping_file in json_mapping_files:
        base_filename = os.path.basename(mapping_file)

        if os.path.isfile(out_descriptions_dir +"/"+base_filename):
            print(f"Skipping... Descriptions File[{base_filename}] already exists")
            continue

        with open(mapping_file, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            # Adapt to respective JSON structure
            # mapping_keys.add(in_json.get("original_uri", ""))
            # { "original_uri": str, "response_file_path": str, "document": str}

            response_file_path = in_json.get("response_file_path", "")
            if response_file_path == "":
                print("No response file path found")
                continue
            if not os.path.isfile(response_file_path):
                print(f"Response file path does not exist: {response_file_path}")
                continue
            # Load the response file
            try:
                response_json = utils.get_json_from_response_file(response_file_path)
            except Exception as e:
                print(f"Error loading JSON response file: {e}")
                # Output to an error log file
                error_log_path = response_file_path.replace(".json", "_error.log")
                with open(error_log_path, "a") as error_log_file:
                    error_log_file.write(f"Error loading JSON response file[{response_file_path}]: {e}\n")
            # response_json = utils.get_json_from_response_file(response_file_path)
            # Get the mentions from the response
            try:
                error_spot = "input_text"
                gen_text_w_placeholders = get_info_from_response_json("input_text", response_json, response_file_path)
                error_spot = "mentions"
                original_mentions = get_info_from_response_json("mentions", response_json, response_file_path)
                error_spot = "replacements"
                gen_replacements = get_info_from_response_json("replacements", response_json, response_file_path)
            except Exception as e:
                print(f"Error getting {error_spot} from response file: {e}")
                # Output to an error log file
                error_log_path = response_file_path.replace(".json", "_error.log")
                with open(error_log_path, "a") as error_log_file:
                    error_log_file.write(f"Error loading JSON response file[{response_file_path}]: {e}\n")

            #entities = [[(m) for m in e] for e, key in enumerate(replacements)]
            #print(entities)
            mention_entities = [[(m['new'], prefix_dbpedia + m['wiki']) for m in e['entities']] for e in gen_replacements]
            # Let's already do a query for this document and its alternatives
            uris_to_query = list(set([entry[1] for mention_entity in mention_entities for entry in mention_entity]))
            print(uris_to_query)

            # Step of 200 would hopefully work
            step = 200
            file_counter_desc = 0

            for i in range(0, len(uris_to_query), step):
                print(f"[{response_file_path}] Working on range[{i}, {i+step}]")
                if len(uris_to_query) > step:
                    desc_file_name = base_filename.replace(".json", "")+"_"+str(i)
                else:
                    desc_file_name = base_filename.replace(".json", "")
                # Query DBpedia for descriptions
                dict_uri_desc = sparqlutils.query_descriptions_multiple_uris(uris_to_query[i:i+step])
                candgenutils.save_cand_desc_result(cand_desc_dictionary=dict_uri_desc, file_counter=desc_file_name, result_directory=out_descriptions_dir)
                file_counter_desc += 1
            
            # Check if all entities are valid and exist in DBpedia by querying information
            # Gather all URIs
            #entities = [entity.replace("http://dbpedia.org/resource/", "") for entity in entities]
            #entities = [entity.replace("http://en.wikipedia.org/wiki/", "") for entity in entities]






['Senegal', 'DAKAR', 'Senegal', 'Kaolack', 'Reuters', 'Senegalese', 'Dakar']


In [9]:
# Step 1: Get all mentions
# Step 2: Then generate candidates --> candgenutils.generate_candidates(mention)
# Step 1: Generate NIF dataset
# Step 1.1: Available data:
#   Entrypoint: Original mentions: AIDA-YAGO2-dataset.tsv_nif
#   New mentions: AIDA-YAGO2-dataset.tsv_nif_options/0_response.json
# Step 2: 
# Step 3: Generate candidates and descriptions by calling compute_save_candidates_for_mentions(result_directory, NIFCollection)
# Generate descriptions + save files --> generate_candidates_and_descriptions


import random, re
from pynif import NIFCollection

seed = 42
random.seed(seed)
# for each mapping file, get response_file_path
# take .get("original_uri").replace("https://aifb.kit.edu/conll/", f"https://aifb.kit.edu/conll/synthetic/{seed}/")
#mylist = ["apple", "banana", "cherry"]
#print(random.choice(mylist))
# 

nif_dataset_output_path = f"/mnt/webscistorage/wf7467/agnos/data/"+nif_dataset_name+"_generated_dataset_{seed}.nif"
institute_uri = "https://anonymous.com/"
llm_name = 'deepseek'
original_base_uri = "https://anonymised.com/conll/"
base_uri = original_base_uri + f"synthetic/{seed}/"
prefix_dbpedia = "http://dbpedia.org/resource/"

if STAGE == "generate nif":

    collection = NIFCollection(uri=base_uri)


    def replace_mention_at_index(input_text, mention, replacement, begin_index):
        # Ensure the mention exists at the specified begin_index
        if input_text[begin_index:begin_index + len(mention)] == mention:
            # Replace the mention at the specified index
            input_text = (
                input_text[:begin_index] + replacement + input_text[begin_index + len(mention):]
            )
        else:
            raise ValueError(f"Mention '{mention}' (instead '{input_text[begin_index:begin_index + len(mention)]}') not found at index {begin_index} in the input text.")
        return input_text

    def compute_new_start_indices(desc_order_mentions_original):
        # Compute the new start indices for the mentions in the modified text
        #print("desc_order_mentions_original: ", desc_order_mentions_original)
        new_start_indices = []
        for i, tpl in enumerate(desc_order_mentions_original):
            begin_index = tpl[1]
            modifier = 0
            for j in range(i+1, len(desc_order_mentions_original)):
                prior_tpl = desc_order_mentions_original[j]
                # Get the replacement mention
                replacement = prior_tpl[2]
                # Get the old mention
                old_mention = prior_tpl[0]
                # Compute the new start index
                modifier += len(replacement) - len(old_mention)
            new_start_index = begin_index + modifier
            new_start_indices.append(new_start_index)
        #print(f"New start indices: {new_start_indices}")
        return new_start_indices

    def inject_mentions_into_original_text(original_document, original_mentions, original_beginIndices, original_endIndices, original_entities, mention_entities, gen_text_w_placeholders):

        #original_mentions = [mention_entity[0] for mention_entity in original_mention_entities]
        #original_beginIndices = [mention_entity[1] for mention_entity in original_mention_entities]
        #original_endIndices = [mention_entity[2] for mention_entity in original_mention_entities]
        #original_entities = [mention_entity[3].replace("http://dbpedia.org/resource/", "").replace("http://en.wikipedia.org/wiki/", "") for mention_entity in original_mention_entities]

        new_mentions = [mention_entity['new'] for mention_entity in mention_entities]
        new_entities = [mention_entity['wiki'] for mention_entity in mention_entities]

        if len(original_mentions) == len(mention_entities):
            # same number of mentions in original and generated text
            # We can just replace the mentions with the generated ones

            # Iterate from last index to first index and replace the mentions
            desc_order_mentions_original = [
                (mention, beginIndex, new_mention, original_entity, new_entity) for beginIndex, mention, new_mention, original_entity, new_entity in sorted(
                    zip(original_beginIndices, original_mentions, new_mentions, original_entities, new_entities),
                    key=lambda x: x[0],  # Sort by original_beginIndices
                    reverse=True         # Descending order
                )
            ]

            modified_text = original_document
            for i, tpl in enumerate(desc_order_mentions_original):
                mention = tpl[0]
                begin_index = tpl[1]
                # Get the replacement mention
                replacement = tpl[2]
                print("Tuple: ",tpl)
                try:
                    # Replaces only if mention is exactly there at the right start index
                    modified_text = replace_mention_at_index(modified_text, mention, replacement, begin_index)
                    # How can we compute the new begin index once we have gone through all of them?

                except ValueError as e:
                    raise e
                    # Could not find string at the right spot, let's see if we can find it somewhere though, it might be a faulty index
                    new_begin_index = modified_text.rindex(mention)
                    # Attempt it again - if this doesn't work, then it shouldn't work
                    modified_text = replace_mention_at_index(modified_text, mention, replacement, new_begin_index)

            new_start_indices = compute_new_start_indices(desc_order_mentions_original)

            # # Verifying if new index computation works:
            # for i, idx in enumerate(new_start_indices):
            #     new_start_index = idx
            #     # Get the replacement mention
            #     replacement = desc_order_mentions_original[i][2]
            #     # Get the old mention
            #     old_mention = desc_order_mentions_original[i][0]
            #     # Compute the new start index
            #     print(f"New start index: {new_start_index} for {replacement} (old: {old_mention}) - Verification: {modified_text[new_start_index: new_start_index + len(replacement)]}")

            # Now actually inject the new indices into the data structure
            desc_order_mentions_updated = [(tpl[0], new_start_indices[i], tpl[2], tpl[3], tpl[4]) for i, tpl in enumerate(desc_order_mentions_original)]

            # sort it in ascending order by begin index again
            asc_order_mentions_updated = sorted(
                desc_order_mentions_updated,
                key=lambda x: x[1]  # Sort by new start indices
            )
            # returns the new text as well as the old mentions, start indices, new mentions
            # asc_order_mentions_updated: (mention, beginIndex, new_mention, original_entity, new_entity)
            return modified_text, asc_order_mentions_updated


        elif len(original_mentions) < len(mention_entities):
            # FIXABLE
            # Modified text has more mentions than original text
            # --> Remove them from the modified text

            # Identify which ones correspond to the original mentions
            # and which ones are hallucinated
            # hallucinated mentions should also be replaced but not listed as "mentions"

            # Let's check if the generated mentions are in the original text
            iter_matches = re.finditer(r"\{\d{1,3}\}", gen_text_w_placeholders)
            patched_text = ""
            prev_index = 0
            for i, match in enumerate(iter_matches):
                patched_text += gen_text_w_placeholders[prev_index : match.start()]
                patched_text += new_mentions[i]
                prev_index = match.end()
                #: match.end()]
            print("Patched up text:", patched_text)

            #return None, None

            raise ValueError(f"More generated than wanted mentions, orig:{len(original_mentions)} vs. gen:{len(mention_entities)}: Modified text does not represent the original text well: {gen_text_w_placeholders}")
        elif len(original_mentions) > len(mention_entities):
            raise ValueError(f"[SADNESS] Gen. text does not have enough mentions: modified:({gen_text_w_placeholders}), original:({original_document})")


    def fix_entity(entity):
        # Fix the entity by removing the prefix
        entity = entity.replace("http://dbpedia.org/resource/", "").replace("http://en.wikipedia.org/wiki/", "")
        entity = entity.replace('Toys_"R"_Us', 'Toys_R_Us')
        return entity


    # Load mappings
    # Get all JSON files in the directory
    json_mapping_files = glob.glob(os.path.join(out_mappings_info_dir, "*.json"))
    skip_counter = 0

    for mapping_file in json_mapping_files:
        base_filename = os.path.basename(mapping_file)
        with open(mapping_file, encoding='utf-8') as json_file:
            in_json = json.load(json_file)
            # Adapt to respective JSON structure
            # mapping_keys.add(in_json.get("original_uri", ""))
            # { "original_uri": str, "response_file_path": str, "document": str}
            response_file_path = in_json.get("response_file_path", "")
            # We need a URI or else it shouldn't work
            original_uri = in_json.get("original_uri", "")

            # Original document
            original_document = in_json.get("document", "")

            original_mention_entities = in_json.get("mention_entities", "")

            if original_uri == "":
                print("No original URI found")
                continue
            if response_file_path == "":
                print("No response file path found")
                continue
            if not os.path.isfile(response_file_path):
                print(f"Response file path does not exist: {response_file_path}")
                continue
            # Load the response file
            try:
                response_json = utils.get_json_from_response_file(response_file_path)
            except Exception as e:
                print(f"Error loading JSON response file: {e}")
                # Output to an error log file
                error_log_path = response_file_path.replace(".json", "_error.log")
                with open(error_log_path, "a") as error_log_file:
                    error_log_file.write(f"Error loading JSON response file[{response_file_path}]: {e}\n")

            # Get the mentions from the response
            try:
                error_spot = "input_text"
                gen_text_w_placeholders = get_info_from_response_json("input_text", response_json, response_file_path)
                #error_spot = "mentions"
                #gen_original_mentions = get_info_from_response_json("mentions", response_json, response_file_path)
                error_spot = "replacements"
                gen_replacements = get_info_from_response_json("replacements", response_json, response_file_path)
            except Exception as e:
                print(f"Error getting {error_spot} from response file: {e}")
                # Output to an error log file
                error_log_path = response_file_path.replace(".json", "_error.log")
                with open(error_log_path, "a") as error_log_file:
                    error_log_file.write(f"Error loading JSON response file[{response_file_path}]: {e}\n")
                continue


            ########## Generated information #################
            # Choose a random set of entities as replacements
            replacement_entities = random.choice(gen_replacements)
            mention_entities = replacement_entities.get("entities")

            ########## Original information #################
            # Sort the original mentions and entities
            original_mentions_entities_sorted = sorted(
                [(mention_entity[0], mention_entity[1], mention_entity[2], mention_entity[3]) for mention_entity in original_mention_entities],
                key=lambda x: x[1]
            )

            original_mentions =     [mention_entity[0] for mention_entity in original_mentions_entities_sorted]
            original_beginIndices = [mention_entity[1] for mention_entity in original_mentions_entities_sorted]
            original_endIndices =   [mention_entity[2] for mention_entity in original_mentions_entities_sorted]
            original_entities =     [mention_entity[3].replace("http://dbpedia.org/resource/", "").replace("http://en.wikipedia.org/wiki/", "") for mention_entity in original_mentions_entities_sorted]


            try:
                generated_text, ordered_updated_mention_beginIndex_entities = inject_mentions_into_original_text(original_document, original_mentions, original_beginIndices, original_endIndices, original_entities, mention_entities, gen_text_w_placeholders)
            except Exception as e:
                skip_counter += 1
                print("Skip Counter: ", skip_counter)
                print(f"Error injecting mentions into original text: {e}")
                continue
            print("[ORIG TEXT]", original_document)
            print(f"[GEN TEXT] {generated_text}")

            # if len(original_mentions) == len(mention_entities):
            #     if all_indices_found:
            #         # Found all indices in generated text (for the generated mentions) 
            #         # AND the mention counts match up with the original mention count

            #         # There is consistency in the number of generated mentions and the placeholders in the generated placeholder text
            #         # so we can technically just replace the mentions with the generated ones

            #         # Replace placeholders from left to right and get the right begin indices and end indices (=begin index + mention length)
            #         # Assume "everything is fine" in the generated text and mentions (incl. mention order)
            #         beginIndices = []
            #         endIndices = []
            #         new_mentions = []
            #         new_entities = []

            #         # Assumption: mention_entities is sorted from left to right in order of occurence of {0}, {1}, {2} etc.
            #         # Otherwise index calculation is faulty (e.g. in the case of {5}, {4}, {6})
            #         for i, e in enumerate(mention_entities):
            #             to_replace = "{"+str(i)+"}"
            #             new_mention = e['new']
            #             new_entity = e['wiki']
            #             if to_replace in input_text:
            #                 # Text contains {i} that we want to replace...
            #                 start_index = input_text.index(to_replace)
            #                 end_index = start_index + len(new_mention)
            #                 input_text = input_text.replace(to_replace, new_mention, 1)
            #                 beginIndices.append(start_index)
            #                 endIndices.append(end_index)
            #                 new_mentions.append(new_mention)
            #                 new_entities.append(new_entity)
            #             else:
            #                 # Text does not contain {i} that we want to replace with actual
            #                 raise ValueError(f"[Replacement/Index retrieval] {to_replace} not found (M[{new_mention}]: E[{new_entity}]) in input text: {input_text}")
            #     else:
            #         # {i} is not consistent in the generated text, so we have to do the replacement manually
            #         # Same number of mentions though, so that's not an issue
            #         # It's not consistent, meaning we have to rely on the original text document
            #         generated_text = inject_mentions_into_original_text(original_document, original_mentions, original_beginIndices, original_endIndices, original_entities, mention_entities)

            # else:
            #     if len(original_mentions) < len(mention_entities):
            #         # FIXABLE
            #         # Modified text has more mentions than original text
            #         # --> Remove them from the modified text
            #         generated_text = inject_mentions_into_original_text(original_document, original_mentions, original_beginIndices, original_endIndices, original_entities, mention_entities)

            #     elif len(original_mentions) > len(mention_entities):
            #         # Modified text does not have enough mentions 
            #         # --> needs manual proofreading
            #         raise ValueError(f"Synthetic text does not have enough mentions: synthetic:({len(mention_entities)}), original:({len(original_mentions)})")



            # sorted_replacement_indices = sorted(replacement_indices)
            # # Check if the replacement indices are sorted - otherwise our indexing logic is faulty
            # if replacement_indices != sorted_replacement_indices:
            #     # Assumption: mention_entities is sorted from left to right in order of occurence of {0}, {1}, {2} etc.
            #     # Otherwise index calculation is faulty (e.g. in the case of {5}, {4}, {6})
            #     raise ValueError(f"Replacement indices are not sorted: {replacement_indices} in input text: {gen_text_w_placeholders}")

            # Finished loading everything we need, so let's create the NIF documents now
            # Generate a document URI incl. seed etc.
            document_uri = original_uri.replace(original_base_uri, base_uri)
            
            sentence_str = generated_text

            # Check if the sentence actually contains all the mentions at the right spots now
            for i, m in enumerate(ordered_updated_mention_beginIndex_entities):
                # asc_order_mentions_updated: (mention, beginIndex, new_mention, original_entity, new_entity)
                beginIndex = m[1]
                new_mention = m[2]
                endIndex = beginIndex + len(new_mention)
                substring_mention = sentence_str[beginIndex:endIndex]
                if substring_mention != new_mention:
                    raise ValueError(f"New mention '{new_mention}' vs. '{substring_mention}' not found  at [{beginIndex}, {endIndex}] in sentence:{sentence_str}")

            # Add sentence (document) to context
            context = collection.add_context(
            uri=document_uri,
            mention=sentence_str)

            # For each mention
            for i, e in enumerate(ordered_updated_mention_beginIndex_entities):
                # asc_order_mentions_updated: (mention, beginIndex, new_mention, original_entity, new_entity)
                entity = fix_entity(e[4])
                # Add an entry
                context.add_phrase(
                beginIndex=e[1],
                endIndex=e[1]+len(e[2]),
                #taClassRef=['http://dbpedia.org/ontology/SportsManager', 'http://dbpedia.org/ontology/Person', 'http://nerd.eurecom.fr/ontology#Person'],
                #score=0.9869992701528016,
                annotator=institute_uri + "synthetic/" + llm_name,
                taIdentRef='http://dbpedia.org/resource/' + entity,
                #taMsClassRef='http://dbpedia.org/ontology/SoccerManager'
                )
                print("Adding to context: ", e)
        print(f"Finished processing file: {mapping_file}: Skipped[{skip_counter}]")
            #mention_entities = [[(m['new'], prefix_dbpedia + m['wiki']) for m in e['entities']] for e in replacements]

    generated_nif = collection.dumps(format='turtle')
    with open(nif_dataset_output_path, "w", encoding='utf-8') as dataset_file:
        dataset_file.write(generated_nif)



In [10]:
# Move this to a python script...
import candgenutils
import utils

# Generate candidates for the new mentions

out_candidates_dir = "./data/"+nif_dataset_name+"_candidates/"#"C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/"
in_collection_path = nif_data_path#nif_dataset_output_path#"./data/"+"synthetic_aida_dataset_42.nif"
if STAGE == "generate candidates":
    print("Loading NIF dataset from: ", in_collection_path)
    syn_collection = utils.load_nif_dataset(in_collection_path)

    print("Generating candidates for new mentions...")
    candgenutils.compute_save_candidates_for_mentions(result_directory=out_candidates_dir, collection=syn_collection)
    print("Finished generating candidates for new mentions.")

Loading NIF dataset from:  /mnt/webscistorage/wf7467/agnos/data/AIDA-Syn_mini_10_42.nif
Finished reading NIF data from file(/mnt/webscistorage/wf7467/agnos/data/AIDA-Syn_mini_10_42.nif).
<NIFCollection http://anon.ymous/>
Generating candidates for new mentions...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

ValueError: Could not retrieve candidates for mention[Brazilian], code: 503

In [ ]:
print("Hello worldasd")

Hello worldasd


In [ ]:
        
# All the entities that are not in description files are not correct entities
# -> see how many documents such things affect
#  


# Generate candidates for the new mentions via DBpedia Lookup or BLINK
# Generate types for each candidate via DBpedia Lookup or SPARQL query to DBpedia endpoint
import sparqlutils
import candgenutils


base_path = nif_data_folder_path
out_candidates_dir = base_path + nif_dataset_name + "_candidates/"#"C:/Users/wf7467/Desktop/GitHub/KIT/agnostic-disambiguation/data/candidates/CoNLL_AIDA-YAGO2-dataset.nif/"
out_candidates_descriptions_dir = base_path + nif_dataset_name + "_candidate_descriptions/"

if STAGE in ["check for missing descriptions", "generate descriptions"]:
    # Load existing ones
    all_candidate_results = candgenutils.load_candidate_results(result_directory=out_candidates_dir)
    print(f"Loaded {len(all_candidate_results)} document entries")


    # For the rest: generate candidates

    # save_candidate_result
    # transform_dbp_json_cands_to_dict: DBpedia JSON to dict
    # Step 0: Generate candidates for mentions
    # Step 1: Collect all relevant URIs 
    # Step 2: Query descriptions for all URIs
    # Step 3: Save the results

    # Step 1: For which URIs do we need additional info (aka. descriptions)?
    wanted_keys = ['candidate_uris', 'true_uris']
    lst_len_counter = 0
    uris_for_descriptions = set()

    for in_file in all_candidate_results:
        entry = all_candidate_results[in_file]
        #print(in_file, entry.keys())

        for document in entry:
            document_candidate_json = entry[document]
            #print(f"Document cand. json: {document_candidate_json}")
            for key in wanted_keys:
                #print(f"Key: {key}")
                uris_of_uris = document_candidate_json[key]
                #print(f"URIs of URIs: {type(uris_of_uris[0])}")
                # For some reason URIS_of_URIS is sometimes a list of strings rather than list of lists - should debug
                if uris_of_uris == []:
                    print(f"Empty list for [{key}] in [{document}]")
                    continue
                if isinstance(uris_of_uris[0], str):
                    # It's only one entry, so add it.
                    uris_for_descriptions.update(uris_of_uris)
                elif isinstance(uris_of_uris[0], list):
                    for uris in uris_of_uris:
                        lst_len_counter = lst_len_counter + len(uris)
                        uris_for_descriptions.update(uris)
                        if ")" in uris:
                            print(f"Found 'h' among {len(uris)} URIs in [{key}] for [{document}]: {uris}")
                    #print(len(uris_for_descriptions))
                    #print(lst_len_counter)
            #break
        #break

    print(lst_len_counter)
    print(len(uris_for_descriptions))
    print(next(iter(uris_for_descriptions)))
    uris_for_descriptions = list(uris_for_descriptions)


if STAGE == "generate descriptions":
    # Step 2: Query descriptions for all URIs & save results
    step = 100
    file_counter_desc = 0
    for i in range(0, len(uris_for_descriptions), step):
        print("Working on range[%d, %d]" % (i, i+step))
        try:
            dict_uri_desc = sparqlutils.query_descriptions_multiple_uris(uris_for_descriptions[i:i+step])
            # Step 3: Save results
            candgenutils.save_cand_desc_result(cand_desc_dictionary=dict_uri_desc, file_counter=i, result_directory=out_candidates_descriptions_dir)
            file_counter_desc += 1
        except Exception as e:
            print(f"Error querying descriptions: {uris_for_descriptions[i:i+step]}")
            continue


In [ ]:
if STAGE == "check for missing descriptions":
        
    # Repeatedly check that everything is where it is supposed to be
    # Checking 3 times for missing URIs... if they continue missing, it means they don't have an abstract. Most are due to being disambiguation pages
    # Example of missing URIs:
    # ['http://dbpedia.org/resource/Paul_Whelan', 'http://dbpedia.org/resource/Rovereto_(disambiguation)', 'http://dbpedia.org/resource/Irwin', 'http://dbpedia.org/resource/CEFCU', 'http://dbpedia.org/resource/Angolan_swallow', 'http://dbpedia.org/resource/Mountain_View,_New_Mexico', 'http://dbpedia.org/resource/Siege_of_Damascus', 'http://dbpedia.org/resource/Jerry_Simmons', 'http://dbpedia.org/resource/Frank_Richter', 'http://dbpedia.org/resource/CGU', 'http://dbpedia.org/resource/El_Gran_campeón', 'http://dbpedia.org/resource/Tsentaroy', 'http://dbpedia.org/resource/DoÃ±a_Ana_County,_New_Mexico', 'http://dbpedia.org/resource/Bethlehem_High_School', 'http://dbpedia.org/resource/List_of_unnumbered_minor_planets:_2003_A–R', 'http://dbpedia.org/resource/SPAP', 'http://dbpedia.org/resource/Charleston_Civic_Center', 'http://dbpedia.org/resource/ASAC', 'http://dbpedia.org/resource/Robert_Irwin', 'http://dbpedia.org/resource/Maarit_LepomÃ¤ki', 'http://dbpedia.org/resource/PloieÈ™ti', 'http://dbpedia.org/resource/Quattro_Canti,_Catania', 'http://dbpedia.org/resource/Peruvian_Brazilians', "http://dbpedia.org/resource/Church_of_Sant'Angelo,_Perugia", 'http://dbpedia.org/resource/Yelena_Vyalbe', 'http://dbpedia.org/resource/Saint-cyriens', 'http://dbpedia.org/resource/1994_Milan_Indoor_–_Singles', 'http://dbpedia.org/resource/Tuqtu', 'http://dbpedia.org/resource/Military_of_Mauritius', 'http://dbpedia.org/resource/Alistar_Fredericks', "http://dbpedia.org/resource/2002_Kroger_St._Jude_International_–_Women's_Doubles", 'http://dbpedia.org/resource/Wayne_Simmons', 'http://dbpedia.org/resource/Lindsay_(name)', 'http://dbpedia.org/resource/List_of_freshman_class_members_of_the_111th_United_States_Congress', 'http://dbpedia.org/resource/Neohumanism', 'http://dbpedia.org/resource/Kenan_IÅŸÄ±k', 'http://dbpedia.org/resource/Belfast_Duncairn', 'http://dbpedia.org/resource/Hum_Sab_Chor_Hain', 'http://dbpedia.org/resource/Flamengo_(disambiguation)', 'http://dbpedia.org/resource/Hà_Tĩnh_Province', 'http://dbpedia.org/resource/List_of_KDE_applications', 'http://dbpedia.org/resource/Sandusky_(surname)', 'http://dbpedia.org/resource/Kishibe', 'http://dbpedia.org/resource/Toyota_concept_vehicles_(1970–1979)', 'http://dbpedia.org/resource/FranÃ§ois_PÃ©ron', 'http://dbpedia.org/resource/WGCL-TV', "http://dbpedia.org/resource/2017_Ricoh_Open_–_Women's_Doubles", 'http://dbpedia.org/resource/KB_Financial_Group_Inc', 'http://dbpedia.org/resource/Arjun_Singh_(politician,_born_1962)', 'http://dbpedia.org/resource/103rd_Infantry_Division_Piacenza']
    import candgenutils

    # Check for missing descriptions
    candgenutils.check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=47, prefix="missing_l_", result_directory=out_candidates_descriptions_dir)
    candgenutils.check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=23, prefix="missing_m_", result_directory=out_candidates_descriptions_dir)
    candgenutils.check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=13, prefix="missing_n_", result_directory=out_candidates_descriptions_dir)
    candgenutils.check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=3, prefix="missing_o_", result_directory=out_candidates_descriptions_dir)
    candgenutils.check_for_missing_uri_descriptions_and_complete(uris_for_descriptions=uris_for_descriptions, step=1, prefix="missing_p_", result_directory=out_candidates_descriptions_dir)